# Import

In [34]:
import pandas as pd
import numpy as np

# from IPython.display import display
# pd.options.display.max_columns = 500
# pd.options.display.max_rows = 500

%matplotlib inline

In [35]:
train_raw = pd.read_csv('data/train.csv')

In [36]:
train_df = pd.read_csv('data/train.csv')
train_df = train_df.drop('Id', axis = 1)

In [37]:
test_df = pd.read_csv('data/test.csv')

In [38]:
y = train_df['SalePrice']
train_df = train_df.drop('SalePrice', axis=1)

# Divide varbles by type

In [39]:
# date variables
date_vars = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

# list comprehension of continuous variables that are not dates
cont_vars = [cont_var for cont_var in train_df.columns if
        (train_df[cont_var].dtype == 'float64') | (train_df[cont_var].dtype == 'int64')
        and not cont_var in date_vars]

# change continus varibles to float
train_df[cont_vars] = train_df[cont_vars].astype('float64')

# create dataframe of date variables
dates = train_df[date_vars].astype(object)

In [40]:
# dict compehension of the frequncy of unique varibales
cont_freq = {cont_var: train_df[cont_var].nunique() for cont_var in cont_vars}
# frequency data frame
cont_freq_df = pd.DataFrame.from_dict(cont_freq, orient='index').rename(columns={0: 'freq'})
# continuous variables with few unique values
low_vol_cont = ['LowQualFinSF', '3SsnPorch', 'PoolArea']
# variabkes with Nan as the mode
mode_na = ['Alley', 'PoolQC', 'Fence','MiscFeature', 'FireplaceQu']


In [41]:
# data frame with numerical variables 
nums = train_df[list(cont_freq_df[cont_freq_df.freq >= 50].index)].astype(object)

num_cats = train_df[list(cont_freq_df[cont_freq_df.freq < 50].index)].astype(object)

# list of non categorical variables
non_cat = list(nums.columns) + list(dates.columns) + list(num_cats.columns) + mode_na
# data frame with only categorical
cats =  train_df.drop(non_cat, axis = 1)

# data frame with categoricals where the mode is nan
cats_na = train_df[mode_na]

# move continuous variables with low-frequency from num_cats to nums
nums[low_vol_cont] = num_cats[low_vol_cont]
num_cats = num_cats.drop(low_vol_cont, axis=1)

# Fill numarical missing with 0

In [42]:
nums = nums.fillna(0)

# Add dummies remove dominate and original categorical columns

In [43]:
def add_dummies_remove_modes(dummy_list, house_raw, house_df):

    dummy_modes = list(house_raw[dummy_list].mode().iloc[0,:].items())

    dummy_modes = [(col, (float(mode))) 
                   if type(mode) == int else (col, mode) for col, mode in dummy_modes]

    drop_modes = list(map(lambda x: str(x[0]) + '_' + str(x[1]), dummy_modes))

    dummy_cols = pd.get_dummies(
        house_df[dummy_list].astype(object), dummy_na=True).drop(drop_modes, axis=1)

    return pd.concat([
       house_df.drop(dummy_list, axis=1),
       dummy_cols], axis=1)


# Numarical Categorical to Dummies

In [44]:
num_cats_raw = num_cats

# impute numarical categoricals with mode
num_cats = num_cats.fillna(num_cats.mode())

# dummify categotical numericals
num_cats = add_dummies_remove_modes(list(num_cats.columns), train_df, num_cats)

# drop columns that are all 0
num_cats = num_cats.drop(
    num_cats.columns[num_cats.sum()==0], axis=1)


# Categorical to Dummies

In [45]:
cats_raw = cats

#cats dummies
cats = add_dummies_remove_modes(list(cats.columns), train_df, cats)

cats = cats.drop(cats.columns[cats.sum()==0], axis=1)

# Dummify columns with NA as mode

In [46]:
cats_na_raw = cats_na
cats_na = pd.get_dummies(cats_na)

cats_na = cats_na.drop(cats_na.columns[cats_na.sum()==0], axis=1)

# Change GarageYrBlt to binary variable

In [47]:
dates['GarageYrBlt'] = ~dates['GarageYrBlt'].isna()
dates['GarageYrBlt'] = dates['GarageYrBlt'].apply(lambda x: sum([x]))

# Concat final data frame

In [48]:
df_train_final = pd.concat([dates, nums, num_cats, cats, cats_na], axis=1)

# Quality Check

In [49]:
df_train_final.shape

(1460, 337)

In [50]:
cols_before_drop = set(df_train_final.columns)
cols_after_drop = set(df_train_final.T.drop_duplicates(keep='first').T.columns)
cols_before_drop - cols_after_drop

{'BldgType_Duplex',
 'BsmtCond_nan',
 'BsmtFinType1_nan',
 'Condition2_RRAe',
 'Exterior2nd_CBlock',
 'GarageCond_nan',
 'GarageFinish_nan',
 'GarageQual_nan',
 'GarageType_nan',
 'TotRmsAbvGrd_14.0'}

In [51]:
cols_after_drop - cols_before_drop

set()

In [52]:
df_train_final = df_train_final.T.drop_duplicates(keep='first').T

In [53]:
df_train_final.shape

(1460, 327)

# Transform Dummies on Test

In [54]:
test_dummy_cols = num_cats_raw.columns.tolist() +  cats_raw.columns.tolist() +  cats_na_raw.columns.tolist()

In [55]:
 test_dummies1 = pd.get_dummies(test_df[test_dummy_cols])

In [56]:
test_df = pd.concat([test_dummies1, test_df], axis=1)

In [57]:
train_cols = set(df_train_final.columns)
test_cols = set(test_df.columns)

test_drop =  list(test_cols - train_cols)
test_unkonwn_dummies = list(train_cols - test_cols)

# Quality Check on Test

In [58]:
test_drop

['HouseStyle',
 'Exterior2nd_CBlock',
 'SaleCondition',
 'ExterCond_TA',
 'LandSlope_Gtl',
 'CentralAir_Y',
 'SaleType',
 'Functional_Typ',
 'LandSlope',
 'BsmtFinType2_Unf',
 'MSSubClass',
 'Electrical',
 'GarageType_Attchd',
 'Exterior1st',
 'ExterQual_TA',
 'HalfBath',
 'Heating',
 'Condition1',
 'Fence',
 'MasVnrType_None',
 'Exterior1st_VinylSd',
 'BsmtFinType1',
 'BedroomAbvGr',
 'Foundation',
 'BsmtQual',
 'GarageQual',
 'Fireplaces',
 'FullBath',
 'KitchenAbvGr',
 'FireplaceQu',
 'RoofMatl_CompShg',
 'HeatingQC',
 'KitchenQual_TA',
 'Utilities',
 'CentralAir',
 'ExterCond',
 'Foundation_PConc',
 'GarageFinish_Unf',
 'Electrical_SBrkr',
 'ExterQual',
 'BsmtFinType1_Unf',
 'MSZoning_RL',
 'Heating_GasA',
 'GarageCars',
 'LotShape_Reg',
 'Id',
 'MiscVal',
 'OverallQual',
 'LotConfig_Inside',
 'KitchenQual',
 'Condition1_Norm',
 'SaleType_WD',
 'BsmtQual_TA',
 'RoofStyle',
 'Street',
 'PavedDrive_Y',
 'LandContour_Lvl',
 'Utilities_AllPub',
 'RoofStyle_Gable',
 'GarageCond',
 'Exte

In [59]:
test_unkonwn_dummies

['MSSubClass_60.0',
 'Fireplaces_3.0',
 'GarageQual_Ex',
 'OverallCond_9.0',
 'Exterior2nd_Other',
 'TotRmsAbvGrd_12.0',
 'RoofMatl_Metal',
 'TotRmsAbvGrd_7.0',
 'MSSubClass_190.0',
 'OverallCond_2.0',
 'BsmtHalfBath_1.0',
 'KitchenAbvGr_0.0',
 'Condition2_RRNn',
 'OverallCond_6.0',
 'MSSubClass_180.0',
 'TotRmsAbvGrd_4.0',
 'FullBath_1.0',
 'Fireplaces_2.0',
 'RoofMatl_ClyTile',
 'TotRmsAbvGrd_3.0',
 'BedroomAbvGr_0.0',
 'BedroomAbvGr_6.0',
 'KitchenAbvGr_3.0',
 'GarageCars_1.0',
 'MSSubClass_85.0',
 'TotRmsAbvGrd_9.0',
 'MSSubClass_40.0',
 'MiscVal_600.0',
 'FullBath_0.0',
 'MiscFeature_TenC',
 'Exterior1st_Stone',
 'OverallQual_4.0',
 'BedroomAbvGr_5.0',
 'Heating_OthW',
 'MiscVal_480.0',
 'MSSubClass_30.0',
 'Electrical_Mix',
 'Utilities_NoSeWa',
 'TotRmsAbvGrd_10.0',
 'Heating_Floor',
 'MiscVal_450.0',
 'MiscVal_400.0',
 'OverallQual_1.0',
 'MiscVal_560.0',
 'MSSubClass_75.0',
 'OverallQual_6.0',
 'GarageCars_3.0',
 'RoofMatl_Membran',
 'MiscVal_1300.0',
 'TotRmsAbvGrd_11.0',
 'Ov

# Drop Cols not in Train

In [60]:
test_df = test_df.drop(test_drop, axis=1)

In [61]:
test_dummies2 = test_df.reindex(columns = test_unkonwn_dummies, fill_value=0)

In [62]:
test_df = pd.concat([test_df, test_dummies2], axis=1)

# Impute missing with 0

In [63]:
test_na_cols = test_df.columns[test_df.isna().sum() > 0]
test_df[test_na_cols] = test_df[test_na_cols].fillna(0)

# Matching order and add response variable

In [64]:
test_df = test_df[df_train_final.columns]

In [65]:
df_train_final['SalePrice'] = train_raw['SalePrice']

# Pickle

In [68]:
df_train_final.to_pickle('train.pkl')
test_df.to_pickle('test.pkl')